In [430]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, \
                            mean_squared_error
from xgboost import XGBRegressor

pd.options.display.max_columns = None

from scripts import (
    load_uci, load_tidepool_dummy, 
    load_so_pump_raw, load_so_pump_clean,
    load_so_cgm    
)

## CGM data

In [431]:
cgm = load_so_cgm()
print(cgm.shape)
cgm.head()

(25285, 14)


,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
9,10,2019-01-14T00:02:42,EGV,NaN,NaN,NaN,iPhone,128,NaN,NaN,NaN,NaN,1943354.0,41W22H
10,11,2019-01-14T00:07:41,EGV,NaN,NaN,NaN,iPhone,129,NaN,NaN,NaN,NaN,1943654.0,41W22H
11,12,2019-01-14T00:12:41,EGV,NaN,NaN,NaN,iPhone,130,NaN,NaN,NaN,NaN,1943954.0,41W22H
12,13,2019-01-14T00:17:42,EGV,NaN,NaN,NaN,iPhone,131,NaN,NaN,NaN,NaN,1944254.0,41W22H
13,14,2019-01-14T00:22:41,EGV,NaN,NaN,NaN,iPhone,132,NaN,NaN,NaN,NaN,1944554.0,41W22H


In [432]:
cgm['Event Subtype'].nunique()

1

In [433]:
cgm['Event Type'].unique()

array(['EGV', 'Calibration'], dtype=object)

### Drop the non-EGV events

In [434]:
cgm = cgm[cgm['Event Type'] == 'EGV']

In [435]:
print('cgm.shape:', cgm.shape)
cgm.head()

cgm.shape: (25170, 14)


,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
9,10,2019-01-14T00:02:42,EGV,NaN,NaN,NaN,iPhone,128,NaN,NaN,NaN,NaN,1943354.0,41W22H
10,11,2019-01-14T00:07:41,EGV,NaN,NaN,NaN,iPhone,129,NaN,NaN,NaN,NaN,1943654.0,41W22H
11,12,2019-01-14T00:12:41,EGV,NaN,NaN,NaN,iPhone,130,NaN,NaN,NaN,NaN,1943954.0,41W22H
12,13,2019-01-14T00:17:42,EGV,NaN,NaN,NaN,iPhone,131,NaN,NaN,NaN,NaN,1944254.0,41W22H
13,14,2019-01-14T00:22:41,EGV,NaN,NaN,NaN,iPhone,132,NaN,NaN,NaN,NaN,1944554.0,41W22H


In [436]:
cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].min(), cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].max()

('2019-01-14T00:02:42', '2019-04-13T09:04:08')

### Remove unnecessary columns

In [437]:
cgm = cgm.drop(columns=['Index', 'Event Type', 'Event Subtype', 'Patient Info', 'Device Info', 
                        'Source Device ID', 'Insulin Value (u)', 'Carb Value (grams)',
                        'Duration (hh:mm:ss)', 'Glucose Rate of Change (mg/dL/min)',
                        'Transmitter Time (Long Integer)', 'Transmitter ID'])
cgm = cgm.rename(mapper={'Timestamp (YYYY-MM-DDThh:mm:ss)': 'Timestamp'},
                         axis=1)
print('cgm.shape:', cgm.shape)
cgm.head()

cgm.shape: (25170, 2)


,Timestamp,Glucose Value (mg/dL)
9,2019-01-14T00:02:42,128
10,2019-01-14T00:07:41,129
11,2019-01-14T00:12:41,130
12,2019-01-14T00:17:42,131
13,2019-01-14T00:22:41,132


In [438]:
cgm['Timestamp'] = cgm['Timestamp'] \
                    .apply(lambda x: 
                        pd.Timestamp(x).to_datetime64())
cgm.head()

,Timestamp,Glucose Value (mg/dL)
9,2019-01-14 00:02:42,128
10,2019-01-14 00:07:41,129
11,2019-01-14 00:12:41,130
12,2019-01-14 00:17:42,131
13,2019-01-14 00:22:41,132


In [439]:
cgm.Timestamp.iloc[cgm.shape[0] - 1] - cgm.Timestamp.iloc[0]

Timedelta('89 days 09:01:26')

### Convert timestamp to int and apply check mean value as an initial guess

In [440]:
print('cgm.shape:', cgm.shape)
cgm['Timestamp'] = (cgm.Timestamp.astype(int)/(10**9)).astype(int)

# Remove Glucose Values that are not numbers and convert them to numbers
cgm = cgm[~cgm['Glucose Value (mg/dL)'].str.contains("[a-zA-Z]").fillna(False)]
cgm['Glucose Value (mg/dL)'] = cgm['Glucose Value (mg/dL)'].astype(float)

print('cgm.shape:', cgm.shape)
cgm.head()

cgm.shape: (25170, 2)
cgm.shape: (25141, 2)


,Timestamp,Glucose Value (mg/dL)
9,1547424162,128.0
10,1547424461,129.0
11,1547424761,130.0
12,1547425062,131.0
13,1547425361,132.0


In [441]:
cgm.dtypes

Timestamp                  int64
Glucose Value (mg/dL)    float64
dtype: object

In [442]:
def split_train_validate(df, target_name, test_fraction=0.2):

    test_size = int(df.shape[0] * test_fraction)
    df_train = df.iloc[:df.shape[0]-test_size,  :]
    df_val   = df.iloc[ df.shape[0]-test_size:, :]
    X_train  = df_train.drop(columns=[target_name])
    y_train  = df_train[target_name]
    X_val    = df_val.drop(columns=[target_name])
    y_val    = df_val[target_name]

    print('X_train.shape:', X_train.shape, 'y_train.shape:', y_train.shape)
    print('X_val.shape:', X_val.shape, 'y_val.shape:', y_val.shape)
    
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = split_train_validate(cgm, 
                                                     'Glucose Value (mg/dL)', 
                                                     0.2)

X_train.shape: (20113, 1) y_train.shape: (20113,)
X_val.shape: (5028, 1) y_val.shape: (5028,)


In [443]:
def baseline(y_train, y_val):
    
    mean_value_train = y_train.mean()
    
    y_pred = [abs(y - mean_value_train) for y in y_val]
    mean_value_ypred = np.average(y_pred)
    mae = np.average([abs(y - mean_value_ypred) for y in y_pred], axis = 0)
    return mae

print('Average of the training y against validation y gives MAE:', baseline(y_train, y_val))
print('Average of the training y against itself gives MAE:', baseline(y_train, y_train))

Average of the training y against validation y gives MAE: 25.52558788655618
Average of the training y against itself gives MAE: 23.73030596294413


### Create column with glucose values 30 minutes into the future

In [444]:
cgm.head()

,Timestamp,Glucose Value (mg/dL)
9,1547424162,128.0
10,1547424461,129.0
11,1547424761,130.0
12,1547425062,131.0
13,1547425361,132.0


In [445]:
def wrangle(df, minutes):
    seconds = minutes * 60
    xs = df['Timestamp'].values
    ys = []
    for i in range(df.shape[0]):
        y = np.interp(xs[i] + seconds,
                      df['Timestamp'],
                      df['Glucose Value (mg/dL)'])
        ys.append(y)
    print('df.shape:', df.shape)
    print('len(ys):', len(ys))
    df[str(minutes) + ' minutes'] = ys
    return df

In [446]:
cgm = wrangle(cgm, 30)
print('cgm.shape:', cgm.shape)
cgm.head()

df.shape: (25141, 2)
len(ys): 25141
cgm.shape: (25141, 3)


,Timestamp,Glucose Value (mg/dL),30 minutes
9,1547424162,128.0,137.0
10,1547424461,129.0,139.0
11,1547424761,130.0,139.0
12,1547425062,131.0,138.0
13,1547425361,132.0,139.0


In [447]:
cgm.tail(10)

,Timestamp,Glucose Value (mg/dL),30 minutes
25284,1555143548,99.0,90.0
25285,1555143848,97.0,88.0
25286,1555144148,96.0,86.0
25287,1555144448,95.0,87.0
25288,1555144748,93.0,87.0
25289,1555145048,92.0,87.0
25290,1555145348,90.0,87.0
25291,1555145648,88.0,87.0
25292,1555145948,86.0,87.0
25293,1555146248,87.0,87.0


### Must lop off the last 30 minutes of data, since we don't have data to predict the last 30 minutes. This is why we're getting a linear interpolated value of 87 for the last few data points

In [448]:
X_train, X_val, y_train, y_val = split_train_validate(cgm, '30 minutes', 0.2)

X_train.shape: (20113, 2) y_train.shape: (20113,)
X_val.shape: (5028, 2) y_val.shape: (5028,)


In [449]:
param_grid = {
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bytree=1, gamma=0, importance_type='gain',
#        learning_rate=0.06, max_delta_step=0, max_depth=2,
#        min_child_weight=1, missing=None, n_estimators=60, n_jobs=1,
#        nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
#        reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
#        subsample=1)
# gave us +/- 18.66 mg/dL
    
    'learning_rate': [0.05, 0.06, 0.07, 0.08],
    'n_estimators':  [40, 50, 60],
    'max_depth': [2],
#     'subsample': [0.5, 0.75, 0.9],
#     'colsample_bytree': [0.1, 0.2, 0.3, 0.4],
#     'gamma': [0, 1, 2]
}

gridsearch = GridSearchCV(XGBRegressor(),
                          param_grid=param_grid, 
                          # scoring='roc_auc', 
                          cv=3, n_jobs=-1,
                          return_train_score=True, verbose=10)

In [450]:
gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  25 out of  36 | elapsed:    3.3s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  29 out of  36 | elapsed:    3.3s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed:    3.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.5s finished
/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.05, 0.06, 0.07, 0.08], 'n_estimators': [40, 50, 60], 'max_depth': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [451]:
gridsearch.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.06, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=60, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [452]:
gridsearch.cv_results_['mean_train_score']

array([0.6213709 , 0.71610674, 0.75119816, 0.70554766, 0.75182082,
       0.76605401, 0.74282767, 0.76492707, 0.77093251, 0.75934217,
       0.76999521, 0.77302234])

In [453]:
gridsearch.cv_results_['mean_test_score'].mean()

0.7319854928739806

In [454]:
y_pred = gridsearch.predict(X_val)
print('mae:', mean_absolute_error(y_val, y_pred))
print('rmse:', np.sqrt(mean_squared_error(y_val, y_pred)))

mae: 18.663473640513125
rmse: 25.94671565396476


In [455]:
print('len(y_pred):', len(y_pred))

len(y_pred): 5028


### Write out the model and the related data to the filesystem

In [457]:
with open('diabetesmanager/data/private/model', 'wb') as f:
    pickle.dump(gridsearch, f)

In [458]:
cgm.head(30)

,Timestamp,Glucose Value (mg/dL),30 minutes
9,1547424162,128.0,137.000000
10,1547424461,129.0,139.000000
11,1547424761,130.0,139.000000
12,1547425062,131.0,138.000000
13,1547425361,132.0,139.000000
14,1547425662,133.0,140.006667
15,1547425962,137.0,142.020000
16,1547426261,139.0,148.000000
17,1547426561,139.0,153.980066
18,1547426862,138.0,159.000000


In [471]:
# Calculate the shift for y_pred so it matches the Glucose values
def shift_predictions_concat(df, minutes, y_pred):
    seconds = minutes * 60
    first_timestamp = df.Timestamp.iloc[0]
    timestamp = df.Timestamp.values
    shift = None
    for i in range(df.shape[0]):
        t = timestamp[i] - first_timestamp
        if t < seconds:
            continue
        shift = i
        break

    df2 = df.copy().iloc[shift:, :].dropna()
    
    y_pred2 = y_pred[:df2.shape[0]]
    y_pred = y_pred.reset_index(drop=True)

    df2 = df2.reset_index(drop=True)
    df2 = pd.concat([df2, y_pred2], axis=1, ignore_index=True)
    df2 = df2.drop(df2.columns[2], axis=1)
    df2 = df2.rename(columns={0: 'Timestamp', 
                              1: 'Measured', 
                              3: 'Prediction'})

    return df2

y_pred_all = gridsearch.predict(cgm.drop(columns=['30 minutes'], axis=1))
output = shift_predictions_concat(cgm, 30, pd.Series(y_pred_all))
print('output.shape:', output.shape)

with open('diabetesmanager/data/private/table', 'wb') as f:
    pickle.dump(output, f)

output.shape: (25135, 3)


In [460]:
output.tail()

,Timestamp,Measured,Prediction
25130,1555145048,92.0,100.911797
25131,1555145348,90.0,100.911797
25132,1555145648,88.0,98.500099
25133,1555145948,86.0,98.500099
25134,1555146248,87.0,98.104942


In [461]:
print('mae:', mean_absolute_error(output['Measured'], output['Prediction']))
print('rmse:', np.sqrt(mean_squared_error(output['Measured'], output['Prediction'])))

mae: 18.080026025356627
rmse: 24.95112232650992


### Since we have CGM data from 2019-01-14 thru 2019-04-13, we can take the pump data from this date range as well

## Pump data

In [462]:
pump = load_so_pump_clean()
print(pump.shape)
pump.head()

(4507, 7)


,Datetime,Type,Value,Unit,Description,Other Info,Comment
0,2018-08-19 00:00:00,Insulin Summary,12.6,units,End of Day Basal Total,Basal,NaN
1,2018-08-19 00:00:00,Insulin Summary,16.7,units,Bolus-End of Day General Bolus Total,Bolus,NaN
2,2018-08-19 10:28:00,Basal Insulin,1.3,units,Pod activated.Basal rate set to 1.30 units/hour.,Basal,Pod activated.Basal rate set to 1.30 units/hour.
3,2018-08-19 10:30:00,Basal Insulin,1.0,units,Basal rate set to 1.00 units/hour.,Basal,Basal rate set to 1.00 units/hour.
4,2018-08-19 10:44:00,Bolus Insulin,2.0,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...


In [463]:
pump.Datetime.min(), pump.Datetime.max()

(Timestamp('2018-08-19 00:00:00'), Timestamp('2019-04-12 20:18:00'))

In [464]:
pump.Type.value_counts()

Bolus Insulin      2129
Basal Insulin      1191
Insulin Summary     521
Glucose             494
Pump Alarm          152
Notes                20
Name: Type, dtype: int64

In [465]:
pump[pump.Type == 'Bolus Insulin'][:5]

,Datetime,Type,Value,Unit,Description,Other Info,Comment
4,2018-08-19 10:44:00,Bolus Insulin,2.0,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...
5,2018-08-19 13:02:00,Bolus Insulin,1.2,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...
7,2018-08-19 15:57:00,Bolus Insulin,2.5,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...
8,2018-08-19 16:48:00,Bolus Insulin,2.5,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...
9,2018-08-19 17:45:00,Bolus Insulin,3.0,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...


In [466]:
pump[pump.Type == 'Basal Insulin'][:5]

,Datetime,Type,Value,Unit,Description,Other Info,Comment
2,2018-08-19 10:28:00,Basal Insulin,1.3,units,Pod activated.Basal rate set to 1.30 units/hour.,Basal,Pod activated.Basal rate set to 1.30 units/hour.
3,2018-08-19 10:30:00,Basal Insulin,1.0,units,Basal rate set to 1.00 units/hour.,Basal,Basal rate set to 1.00 units/hour.
10,2018-08-19 18:27:00,Basal Insulin,0.0,units,Basal suspended.Basal rate set to 0.00 units/h...,Basal,Basal suspended.Basal rate set to 0.00 units/h...
12,2018-08-19 18:59:00,Basal Insulin,1.0,units,Basal resumed.Basal rate set to 1.00 units/hour.,Basal,Basal resumed.Basal rate set to 1.00 units/hour.
15,2018-08-19 22:49:00,Basal Insulin,0.0,units,Basal suspended.Basal rate set to 0.00 units/h...,Basal,Basal suspended.Basal rate set to 0.00 units/h...


In [467]:
pump[pump.Type == 'Insulin Summary'][:5]

,Datetime,Type,Value,Unit,Description,Other Info,Comment
0,2018-08-19,Insulin Summary,12.60,units,End of Day Basal Total,Basal,NaN
1,2018-08-19,Insulin Summary,16.70,units,Bolus-End of Day General Bolus Total,Bolus,NaN
17,2018-08-20,Insulin Summary,12.50,units,Bolus-End of Day General Bolus Total,Bolus,NaN
18,2018-08-20,Insulin Summary,20.15,units,End of Day Basal Total,Basal,NaN
34,2018-08-21,Insulin Summary,22.10,units,End of Day Basal Total,Basal,NaN


In [468]:
pump[pump.Type == 'Glucose'][:5]

,Datetime,Type,Value,Unit,Description,Other Info,Comment
6,2018-08-19 14:36:00,Glucose,5.6,mmol/L,NaN,NaN,NaN
40,2018-08-21 10:49:00,Glucose,4.9,mmol/L,NaN,NaN,NaN
65,2018-08-22 14:05:00,Glucose,9.0,mmol/L,NaN,NaN,NaN
95,2018-08-24 20:26:00,Glucose,5.3,mmol/L,NaN,NaN,NaN
109,2018-08-25 12:18:00,Glucose,8.9,mmol/L,NaN,NaN,NaN


In [469]:
pump[pump.Type == 'Pump Alarm'].Description.unique()

array(['OmniPod Alarm: Suspend end advisory alarm',
       'OmniPod Alarm: Pod expiration advisory alarm',
       'OmniPod Alarm: Low reservoir advisory alarm',
       'OmniPod Alarm: Pod expired hazard alarm',
       'OmniPod Alarm: Clock reset alarm', 'OmniPod Alarm: Pod alarm',
       'OmniPod Alarm: Last Pod status hazard alarm'], dtype=object)

In [470]:
pump[pump.Type == 'Notes'].Description.unique()

array(['Date changed to: 10/21/2018', 'Time changed to: 15:05',
       'Time changed to: 19:00', 'Basal rate set to 1.00 units/hour.',
       'General Bolus.Delivered: 1.00.', 'General Bolus.Delivered: 1.50.',
       'Pod deactivated.Basal rate set to 0.00 units/hour.',
       'Pod activated.Basal rate set to 1.00 units/hour.',
       'General Bolus.Delivered: 2.10.', 'General Bolus.Delivered: 1.10.',
       'General Bolus.Delivered: 3.00.', 'General Bolus.Delivered: 2.50.',
       'General Bolus.Delivered: 3.60.', 'General Bolus.Delivered: 1.60.',
       'General Bolus.Delivered: 5.60.',
       'Temporary basal rate set to 0.75 units/hour.Temp percent change: -25%.',
       'Time changed to: 22:14'], dtype=object)

### Use Bolus/Basal Insulin rows, and use data only after 10/21/2018 or before - never across. There was a date/time change on the pump on that day

## Data Cleaning